In [71]:
from gwosc.datasets import event_gps
from gwpy.timeseries import TimeSeries, TimeSeriesDict
from gwpy.frequencyseries import FrequencySeries
from gwpy.signal import filter_design
from gwpy.plot import Plot
from scipy.special import erf
from ipywidgets import interactive_output, VBox, HBox, FloatSlider, Checkbox, FloatLogSlider, FloatRangeSlider, FloatText

import numpy as np
import matplotlib.pyplot as plt
import plotfancy as pf
pf.housestyle_rcparams()


True

In [4]:
gps = event_gps('GW150914')
data = TimeSeries.fetch_open_data('L1', gps-5, gps+5)

In [91]:
def gs(x, centre=0, width=1, softness=0.1):
    return 0.5 * (erf((x - (centre - width/2)) / softness) - erf((x - (centre + width/2)) / softness))

def gslr(x, left, right, softness=0.1):
    return 0.5 * (erf((x - (left)) / softness) - erf((x - (right)) / softness))


gridlims = [1,1000]
res = 1000

def process_and_plot(lr,s, notch_b, notch_fun=60):
    grid = np.arange(gridlims[0], gridlims[1], res)
    
    dat = gslr(grid, lr[0], lr[1],softness=s)

    notch_f = notch_fun*np.linspace(1,5)

    notch_f = [60,180,220]
    if notch_b:
        notch_mask = np.ones(len(grid))
        for notch in notch_f:
            notch_mask = notch_mask*(1-gs(grid, width=2, softness=0.1, centre=notch))
    
        dat = dat*notch_mask

    fig, ax1 = pf.create_plot((4,3))
    ax1.set_xlim(gridlims)
    ax1.set_ylim(-0.05,1.05)
    ax1.plot(grid,dat, color='black', lw=2, zorder=10)
    ax1.set_xscale('log')

    ax2 = ax1.twinx()
    ax2.plot(data.psd(), color='#77aca2', alpha=0.6, zorder=-10)
    ax2.set_yscale('log')
    ax2.set_ylabel('Noise PSD')

    ax1.set_ylabel('$\exp($Gain [dB]$)$')
    ax1.set_xlabel('Frequency [Hz]')
    
    pf.fix_plot([ax1,ax2])

    ax1.tick_params(top=True,right=False, direction='in', length=7, which='major')
    ax1.tick_params(top=True,right=False, direction='in', length=4, which='minor')


    
    plt.show()
    plt.close(fig)

#### - OLD - ####
# l_slider = FloatLogSlider(min=0, max=3, step=0.01, value=np.log10(25), description='BP Left')
# r_slider = FloatLogSlider(min=0, max=3, step=0.01, value=np.log10(300), description='BP Right')

#### - SLIDERS - ####
lr_slider = FloatRangeSlider(min=1, max=1000, step=0.01, value=[24,300], description='BP')
s_slider = FloatLogSlider(min=-1, max=2, step=0.1, value=-.5, description='Softness')

n_box = FloatText(value=60, description='Ground Base Harmonic')
n_checkbox = Checkbox(value=False, description='Notches On?')

#### - PLOTTING - ####
out = interactive_output(process_and_plot, {'lr': lr_slider, 's':s_slider, 'notch_b':n_checkbox, 'notch_fun':n_box})
VBox([lr_slider, s_slider,n_checkbox,n_box, out])



<>:37: SyntaxWarning: invalid escape sequence '\e'
<>:37: SyntaxWarning: invalid escape sequence '\e'
/var/folders/6b/x84gj0ds47n3hjglw1wq1j8m0000gn/T/ipykernel_93347/3088864182.py:37: SyntaxWarning: invalid escape sequence '\e'
  ax1.set_ylabel('$\exp($Gain [dB]$)$')


In [80]:
n=10
harmonic_series = 60*np.cumsum(1 / np.arange(1, n + 1))
harmonic_series

array([ 60.        ,  90.        , 110.        , 125.        ,
       137.        , 147.        , 155.57142857, 163.07142857,
       169.73809524, 175.73809524])

In [86]:
notch_fun = 60
notch_fun*np.arange(1,4)

array([ 60, 120, 180])